In [ ]:
import os
pwd = os.path.abspath('')
SKETCH_169 = os.path.join(pwd, '..', '..', '..', 'sketch3', 'sketch-1.6.9', 'sketch-frontend', 'runtime')
SKETCH_175 = os.path.join(pwd, '..', '..', 'sketch-frontend')
SKETCH_169_PATH = os.path.join(SKETCH_169, '..')
SKETCH_175_PATH = os.path.join(SKETCH_175, 'target', 'sketch-1.7.5-noarch-launchers')
SKETCH_VERSION = '1.7.5'
SKETCH_HOME = SKETCH_175
SKETCH_PATH = SKETCH_175_PATH
os.environ['JAVA_HOME'] = JAVA_HOME = '/usr/lib/jvm/java-1.8.0-openjdk-amd64'
os.environ['PATH'] = JAVA_HOME + '/bin' + os.pathsep + os.environ['PATH']

In [ ]:
import sys; sys.path.insert(0,'./')
from sketch import run_sketch
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import datetime
import json
import math
import multiprocessing

In [ ]:
BENCHMARK = "benchmark"
DATA = "data"

g_opt = type('', (object,), {
    "config": "config.full.json",
    "benchmarks": os.listdir(BENCHMARK),
    "strategies": ["WILCOXON"], # "SINGLE", "NOT_SET", "WILCOXON"
    "cores": [],
    "degrees": [],
    "repeat": 6, # AC paper used 13 
    "timeout": 30,
    "ntimes": None,
    "register": False,
    "vanilla": False,
})

all_benchmark_short_names = [
    ("p_button", "button_demo"),
    ("p_color", "colorchooser_demo"),
    ("p_menu", "menu_demo"),
    ("l_prepend", "list_prepend"),
    ("l_min", "list_min"),
    ("a_mom_1", "shoaib_1"),
    ("a_mom_2", "shoaib_2"),
    ("ar_s_4", "sygus_ar_s_4"),
    ("ar_s_5", "sygus_ar_s_5"),
    ("ar_s_6", "sygus_ar_s_6"),
    ("ar_s_7", "sygus_ar_s_7"),
    ("ar_sum", "sygus_ar_sum"),
    ("hd_13_d5", "sygus_hd_13_d5"),
    ("hd_14_d1", "sygus_hd_14_d1"),
    ("hd_14_d5", "sygus_hd_14_d5"),
    ("hd_15_d5", "sygus_hd_15_d5"),
    ("s_cg", "cg"),
    ("s_log2", "log2"),
    ("s_logcnt", "logcount"),
    ("s_rev", "reverse"),
    ("deriv2", "deriv2"),
    ("deriv3", "deriv3"),
    ("deriv4", "deriv4"),
    ("deriv5", "deriv5"),
    ("q_noti", "qbs_noti"),
    ("q_serv", "qbs_serv")
]

In [ ]:
def real_time(e):
    return (e['after'] - e['before']).total_seconds()

In [ ]:
def mean_siqr(vs):
    return [0.0, 0.0] if len(vs) == 0 else [np.mean(vs), np.subtract(*np.percentile(vs, [75, 25])) / 2]

def time_table(rs, cols):
    def realonly(c):
        return c in ["V", "1"]
    def subcols(c, b):
        es = list(filter(lambda e: e['successful'], rs[c][b]))
        if not es:
            return [np.inf] * (2 if realonly(c) else 4)
        return mean_siqr(list(map(real_time, es))) + ([] if realonly(c) else mean_siqr(list(map(lambda e: e['cpu_time'], es))))
    def subhead(c):
        return ["real", "siqr$_r$"] + ([] if realonly(c) else ["cpu", "siqr$_c$"])
    index, benchmarks = zip(*filter(lambda b: b[1] in config.keys(), all_benchmark_short_names))
    times = [[time for c in cols for time in subcols(c, b)] for b in benchmarks]
    all_cols = [ix for c in cols for ix in
                ([('', 'Sketch Time (s)',   t) for t in subhead(c)] if c == 'V' else
                 [('# Cores (Time (s))', c, t) for t in subhead(c)])]
    real_cols = pd.IndexSlice[:, [ix for ix in all_cols if ix[2] == 'real']]
    siqr_cols = pd.IndexSlice[:, [ix for ix in all_cols if ix[2].startswith('siqr')]]
    columns = pd.MultiIndex.from_tuples(all_cols)
    return pd.DataFrame(times, index=index, columns=columns).style.apply(
        lambda s: ['font-weight: bold; background-color: DarkSeaGreen'
                   if v else '' for v in s == s.min()],
        axis=1,
        subset=real_cols
    ).apply(
        lambda s: ['background-color: Maroon'
                   if v else '' for v in s == s.max()],
        axis=1,
        subset=real_cols
    ).where(
        lambda _: True,
        'border-left: 1px solid',
        subset=real_cols
    ).where(
        lambda _: True,
        'font-size: 66%',
        subset=siqr_cols
    ).format(
        lambda t: '∞' if np.isinf(t) else '%.1f' % t
    )

def speedup_table(rs, cols):
    def real_median(es):
        return np.median(list(map(real_time, es)))
    index, benchmarks = zip(*filter(lambda b: b[1] in config.keys(), all_benchmark_short_names))
    data = [[real_median(rs['V'][b])] +
            [real_median(rs[c][b])/real_median(rs['V'][b]) for c in cols]
            for b in benchmarks]
    van_col = 'Sketch Time (s)'
    ratio_cols = ['Sketch Baseline'] + [c + ' Cores/Sketch' for c in cols if c != 'V']
    columns = [van_col] + ratio_cols
    return pd.DataFrame(data, index=index, columns=columns).style.apply(
        lambda s: ['font-weight: bold; background-color: PaleGreen'
                   if v else '' for v in s == s.min()],
        axis=1,
        subset=ratio_cols
    ).apply(
        lambda s: ['background-color: PaleVioletRed'
                   if v else '' for v in s == s.max()],
        axis=1,
        subset=ratio_cols
    ).format(
        lambda t: '%.1f' % t
    )

In [ ]:
def parallelize(iters, cores_per_iter, runner):
    pool_size = max(1, min(iters, int(multiprocessing.cpu_count()/cores_per_iter)))
    def go(*args, **kwargs):
        return Parallel(n_jobs=pool_size)(delayed(runner)(*args, iteration=i, **kwargs) for i in range(iters))
    return go

In [ ]:
def run_sketch_seed_wrapper(*args, iteration, seed=0, **kwargs):
    return run_sketch(*args, iteration, seed=abs(seed+iteration), **kwargs)
def run(iters, path, main):
    now = int(datetime.datetime.now().strftime("%H%M%S"))
    seed = now * (10 ** int(math.log(iters, 10)))
    return parallelize(iters, 1, run_sketch_seed_wrapper)(
        SKETCH_VERSION, SKETCH_HOME, SKETCH_PATH, path, main,
        mem=32*1024*1024*1024,
        seed=seed,
        timeout=g_opt.timeout,
        noisy=True,
    )

In [ ]:
def fe_p_run(iters, path, main, strategy, core, degree=None):
    return parallelize(iters, core, run_sketch)(
        SKETCH_VERSION, SKETCH_HOME, SKETCH_PATH, path, main,
        par=True, ac=True, degree=degree,
        timeout=g_opt.timeout,
        noisy=True,
        ntimes=g_opt.ntimes, extra=[
        "--slv-strategy", strategy,
        "--slv-p-cpus", str(core),
        "--slv-p-trials", str(32*32*3)]
    )

In [ ]:
def be_p_run(iters, path, main, b, degree, repeat):
    # magic 0.83 number pulled from psketch.p_run
    cores_per_psketch = max(1, int(multiprocessing.cpu_count() * 0.83))
    return parallelize(iters, cores_per_psketch, run_sketch)(
        SKETCH_VERSION, SKETCH_HOME, SKETCH_PATH, path, main,
        mem=4*1024*1024*1024,
        noisy=True,
        ac=True, degree=degree, extra=[
        # custom CEGIS wrapper that runs backend in parallel
        "--fe-cegis-path", os.path.join(pwd, "psketch.py"),
        "--be:conc-benchmark={}".format(b),
        "--be:conc-repeat={}".format(repeat),
        "--be:conc-timeout={}".format(g_opt.timeout),
        "--be:conc-register={}".format(g_opt.register)]
    )

In [ ]:
def run_benchmark(b, benchmark):
    path       = os.path.join(BENCHMARK, b)
    main       = os.path.join(path, benchmark["main"])
    strategies = g_opt.strategies or benchmark["strategies"]
    cores      = g_opt.cores or benchmark["cores"]
    degrees    = g_opt.degrees or benchmark["degrees"]
    iters      = g_opt.repeat

    if g_opt.vanilla: # run vanilla Sketch in parallel
        return run(iters, path, main)

    # use sketch-frontend's parallel running
    def run_strat(strategy):
        if strategy == "SINGLE": # parallel running via wrapper
            return [be_p_run(iters, path, main, b, degree,
                    max(g_opt.cores or [benchmark.get("single_repeat", 1)]))
                    for degree in degrees]
        elif strategy == "NOT_SET": # fixed degree
            return [fe_p_run(iters, path, main, strategy, core, degree)
                    for core in cores
                    for degree in degrees]
        elif strategy == "WILCOXON": # adaptive concretization
            return [fe_p_run(iters, path, main, strategy, core)
                    for core in cores]

    return [r for strategy in strategies
              for rs in run_strat(strategy)
              for r in rs]

In [ ]:
config = json.load(open(g_opt.config))

In [ ]:
SKETCH_VERSION = '1.7.5'
SKETCH_HOME = SKETCH_175
SKETCH_PATH = SKETCH_175_PATH
results_175 = {}

In [ ]:
g_opt.vanilla = True
g_opt.cores = [1]
results_175["V"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
g_opt.vanilla = False
g_opt.cores = [1]
results_175["1"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
g_opt.vanilla = False
g_opt.cores = [2]
results_175["2"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
g_opt.vanilla = False
g_opt.cores = [4]
results_175["4"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
g_opt.vanilla = False
g_opt.cores = [8]
results_175["8"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
df_175_full = time_table(results_175, ('V', '1', '2', '4', '8'))
df_175_full

In [ ]:
df_175_speedup = speedup_table(results_175, ('V', '1', '2', '4', '8'))
df_175_speedup

In [ ]:
df_175_trunc = time_table(results_175, ('V', '1', '4'))
df_175_trunc

In [ ]:
SKETCH_VERSION = '1.6.9'
SKETCH_HOME = SKETCH_169
SKETCH_PATH = SKETCH_169_PATH
results_169 = {}

In [ ]:
g_opt.vanilla = True
g_opt.cores = [1]
results_169["V"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
g_opt.vanilla = False
g_opt.cores = [1]
results_169["1"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
g_opt.vanilla = False
g_opt.cores = [2]
results_169["2"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
g_opt.vanilla = False
g_opt.cores = [4]
results_169["4"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
g_opt.vanilla = False
g_opt.cores = [8]
results_169["8"] = {b: run_benchmark(b, config[b]) for b in config}

In [ ]:
df_169_full = time_table(results_169, ('V', '1', '2', '4', '8'))
df_169_full

In [ ]:
df_169_speedup = speedup_table(results_169, ('V', '1', '2', '4', '8'))
df_169_speedup

In [ ]:
df_169_trunc = time_table(results_169, ('V', '1', '4'))
df_169_trunc